In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import pickle

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)

In [12]:
test_image_caption = pickle.load(open('test_image_caption.pkl', 'rb'))

captions = list(map(lambda x: x['caption'], test_image_caption))

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(captions))
    print(message_embeddings.shape)
    new_test_image_vector = []
    for i, v in enumerate(test_image_caption):
        obj = {}
        obj['name'] = v['name']
        obj['caption'] = v['caption']
        obj['vector'] = message_embeddings[i]
        new_test_image_vector.append(obj)

    with open('test_image_vector.pkl', 'wb') as f:
        pickle.dump(new_test_image_vector, f)

(2000, 512)


In [50]:
from scipy.spatial.distance import cdist
import nltk
from tqdm import tqdm
import csv

test_vectors = pickle.load(open('test_image_vector.pkl', 'rb'))
vectors = list(map(lambda x: x['vector'], test_vectors))

with open('submission.csv', 'w') as csvfile:
    # write csv header
    fieldnames = ['Description_ID', "Top_20_Image_IDs"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    descriptin_test_base = "./all/data/descriptions_test/"
    for file in tqdm(os.listdir(descriptin_test_base)):
        if file.endswith(".txt"):
            file_name = descriptin_test_base + file
            with open(file_name) as f:
                content = f.readlines()
                content = [x.strip() for x in content] 
    #             all_nouns = set()
    #             for l in content:
    #                 nouns = [word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(l)) if pos[0] == 'N']
    #                 for n in nouns:
    #                     all_nouns.add(n)

    #         print(all_nouns)
            print(content[-1])
            with tf.Session() as session:
                session.run([tf.global_variables_initializer(), tf.tables_initializer()])
                message_embedding = session.run(embed([content[-1]])) # parallalize!!
                dist = cdist(message_embedding, vectors, 'cosine')
                print(dist.shape)

            import numpy as np
            sorted_id = np.argsort(dist)[:20][0]
            top_choices =  list(map(lambda x: test_vectors[x]['name'], sorted_id[:20]))
            print(top_choices)
            
            res = {}
            res['Description_ID'] = file
            res['Top_20_Image_IDs'] = " ".join(top_choices)
            writer.writerow(res)

print("Writing Complete.")



"""
# get the nouns of the sentence[five descriptions].
# the images with the correct tag will form first group. others forms the other group. 
"""

"""
Visualize part. 
"""
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# %matplotlib inline
# test_images_base = "./all/data/images_test/"
# for f in top_choices:
#     f_path = test_images_base + f
#     img = mpimg.imread(f_path)
#     plt.figure()
#     plt.imshow(img)
#     plt.show()


  0%|          | 0/2000 [00:00<?, ?it/s]

A giraffe sticking its tongue out over a wooden fence.



  0%|          | 1/2000 [00:22<12:26:08, 22.40s/it]

(1, 2000)
['1448.jpg', '494.jpg', '451.jpg', '478.jpg', '1193.jpg', '1368.jpg', '1826.jpg', '1302.jpg', '1703.jpg', '183.jpg', '1078.jpg', '1469.jpg', '560.jpg', '1832.jpg', '1775.jpg', '415.jpg', '932.jpg', '1534.jpg', '577.jpg', '886.jpg']
A boy holds the guitar controller from Guitar Hero.



  0%|          | 2/2000 [00:46<12:42:31, 22.90s/it]

...


  3%|▎         | 67/2000 [1:08:35<62:01:22, 115.51s/it]

(1, 2000)
['1789.jpg', '343.jpg', '1336.jpg', '982.jpg', '1013.jpg', '844.jpg', '1817.jpg', '1131.jpg', '451.jpg', '541.jpg', '302.jpg', '1834.jpg', '1525.jpg', '1286.jpg', '1803.jpg', '1896.jpg', '1179.jpg', '864.jpg', '1927.jpg', '420.jpg']
A person walking in deep snow and an bus driving through the snow



  3%|▎         | 68/2000 [1:10:48<64:50:58, 120.84s/it]

(1, 2000)
['1195.jpg', '367.jpg', '1933.jpg', '623.jpg', '1311.jpg', '1584.jpg', '1582.jpg', '1885.jpg', '555.jpg', '402.jpg', '1625.jpg', '1333.jpg', '1020.jpg', '489.jpg', '458.jpg', '648.jpg', '217.jpg', '1096.jpg', '767.jpg', '414.jpg']
A homemade pizza with spinach and pears with a slice missing


KeyboardInterrupt: 